<a href="https://colab.research.google.com/github/DarShivank/ChatBot1/blob/main/me.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer(language='english')
import numpy as np
import tensorflow as tf
import random

In [7]:
import json
with open('i.json') as json_data:
    intents = json.load(json_data)

In [8]:
import nltk
nltk.download('punkt')
words = []
classes = []
documents = []
ignore_words = ['?','.', '!', ';']
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

118 documents
30 classes ['Hobbies', 'Identity', 'Shivank', 'activity', 'age', 'appreciate', 'contact', 'covid19', 'cricket', 'datetime', 'exclaim', 'goodbye', 'greeting', 'greetreply', 'haha', 'inspire', 'insult', 'jokes', 'news', 'nicetty', 'no', 'options', 'programmer', 'riddle', 'song', 'suggest', 'thanks', 'timer', 'weather', 'whatsup']
128 unique stemmed words ["'m", "'s", ',', '10', '19', 'a', 'age', 'am', 'anyon', 'are', 'ask', 'awesom', 'bad', 'be', 'best', 'bye', 'can', 'contact', 'could', 'covid', 'creat', 'creator', 'cricket', 'current', 'dar', 'date', 'day', 'design', 'develop', 'do', 'dumb', 'enjoy', 'exist', 'fine', 'for', 'funni', 'get', 'good', 'goodby', 'great', 'haha', 'have', 'he', 'hello', 'help', 'hey', 'hi', 'hobbi', 'hola', 'hot', 'how', 'i', 'idiot', 'india', 'inspir', 'interest', 'is', 'it', 'joke', 'karan', 'know', 'later', 'latest', 'laugh', 'like', 'lmao', 'lol', 'long', 'lost', 'made', 'make', 'match', 'me', 'motiv', 'namast', 'news', 'next', 'nice', 'no',

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:

# create our training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

<ipython-input-9-65cc626bc989>:27: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [10]:
!pip install tflearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127283 sha256=fe458563ad6be93ba35366545817fa3d582ecdeced8539ed390a301795cad83f
  Stored in directory: /root/.cache/pip/wheels/55/fb/7b/e06204a0ceefa45443930b9a250cb5ebe31def0e4e8245a465
Successfully built tflearn


In [11]:
import tflearn

Instructions for updating:
non-resource variables are not supported in the long term


In [12]:
tf.compat.v1.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8, activation='leaky_relu')
net = tflearn.fully_connected(net, 8, activation='leaky_relu')
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net,tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 14999  | total loss: 0.02308 | time: 0.057s
| Adam | epoch: 1000 | loss: 0.02308 - acc: 0.9859 -- iter: 112/118
Training Step: 15000  | total loss: 0.02081 | time: 0.062s
| Adam | epoch: 1000 | loss: 0.02081 - acc: 0.9873 -- iter: 118/118
--


In [13]:

# save all of our data structures
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )


In [16]:
import pickle
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

# import our chat-bot intents file
import json
with open('i.json') as json_data:
    intents = json.load(json_data)

In [17]:
# load our saved model
model.load('./model.tflearn')

In [18]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [19]:
ERROR_THRESHOLD = 0.25
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # a random response from the intent
                    return print(random.choice(i['responses']))

            results.pop(0)

In [20]:
response('do you take cash?')

All good..What about you?


In [33]:
response('')

I am Shivank, just a cooler version


In [35]:
!git clone git@github.com/DarShivank/Chatbot.git

fatal: repository 'git@github.com/DarShivank/Chatbot.git' does not exist
